In [ ]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import altair as alt

import datetime as dt
from scipy.stats import uniform, norm
import pymc3 as pm
import theano
import theano.tensor as tt
import matplotlib.pyplot as plt
import graphviz

from src.utils.preprocessing import MinMaxScaler
from src.model.model import det_dot, drift_model, seasonality_model

In [ ]:
[print(chr(c), end=" ") for c in range(ord("α"), ord("ω")+1)];

## make dummy data to test on

In [ ]:
def dummy_data(t):
    params = dict(
        bias=0.3,
        slope=-0.1,
        factor=-0.2,
        base=3,
        t_shift=0.5,
        t_scale=4.5,
    )
    return drift_func(**params)

def drift_func(**kwargs):
    return (
        kwargs["bias"] + t * kwargs["slope"]
        + kwargs["factor"] * kwargs["base"] ** (kwargs["t_scale"] * (t - kwargs["t_shift"]))
    )

In [ ]:
t = np.arange(100) / 100
y = dummy_data(t)
y += np.random.normal(loc=1, scale=0.1, size=y.shape)
scaler = MinMaxScaler()
y = scaler.fit_transform(y)
df_obs = pd.DataFrame({"t": t, "y": y})

plot_obs = (
    alt.Chart(df_obs).mark_circle(size=3).encode(x="t", y="y", color=alt.value("blue"))
)
plot_obs

## setup model

In [ ]:
with pm.Model() as m:
    
    drift = drift_model(t)
#     yearly = seasonality_model(t)
    yearly=0
    
    σ_ε = pm.Uniform('σ_ε', lower=0, upper=1)
    total = pm.Deterministic("total", drift + yearly)
    
    y_obs = pm.Normal('y_obs', mu=total, sd=σ_ε, observed=y)
    
pm.model_to_graphviz(m)

## test on dummy data

In [ ]:
with m:
    trace = pm.sample(500, tune=500, init="adapt_diag")

In [ ]:
df = pd.DataFrame({"t": t, "y": trace["total"].mean(axis=0)})
plot_hat = alt.Chart(df).mark_circle(size=3).encode(x="t", y="y", color = alt.value("red"))
plot_obs + plot_hat